In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano'
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from keras import backend as K
from tensorflow.python.keras.layers import Layer, InputSpec
from keras import initializers
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


**TextAttnBiRNN**

In [ ]:
!nvidia-smi

Wed Dec  8 09:31:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/gdrive/MyDrive/College/Semester5/ML/Project

/content/gdrive/MyDrive/College/Semester5/ML/Project


In [ ]:
# !pip install --upgrade tensorflow

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import tensorflow as tf

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(name='{}_W'.format(self.name),
                                 shape=(input_shape[-1],),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(name='{}_b'.format(self.name),
                                     shape=(input_shape[1],),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        print(a)
        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

    def cal_att_weights(output, att_w):
        #if model_name == 'HAN':
        eij = np.tanh(np.dot(output[0], att_w[0]) + att_w[1])
        eij = np.dot(eij, att_w[2])
        eij = eij.reshape((eij.shape[0], eij.shape[1]))
        ai = np.exp(eij)
        weights = ai / np.sum(ai)

return weights
from keras import backend as K
sent_before_att = K.function([model1.layers[0].input,K.learning_phase()],  [model1.layers[2].output])
sent_att_w = model1.layers[5].get_weights()
test_seq=np.array(userinp)
test_seq=np.array(test_seq).reshape(1,118,100)
out = sent_before_att([test_seq, 0])

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, Dense, Bidirectional
from keras.layers import GRU
import tensorflow.keras.layers
from tensorflow.keras import layers
class TextAttBiRNN(Model):
    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        super(TextAttBiRNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.bi_rnn = Bidirectional(GRU(128, return_sequences=True))  # LSTM or GRU
        # self.conv_layer = layers.Convolution1D(3, 2, activation="relu")
        self.conv_layer = layers.Convolution1D(30, 2, activation="relu", padding = 'same')
        self.conv_layer2 = layers.Convolution1D(30, 3, activation="relu", padding = 'same')
        # self.pooling_layer = layers.GlobalMaxPool1D(keepdims = True)
        self.input_layer = tf.keras.Input(self.maxlen)
        self.attention = Attention(self.maxlen)
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextAttBiRNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextAttBiRNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        x = self.input_layer(inputs)
        embedding = self.embedding(x)
        x = self.conv_layer(embedding)
        x = self.conv_layer2(x)
        # x = self.pooling_layer(x)
        x = self.bi_rnn(x)
        x = self.attention(x)
        # print(x.shape)
        output = self.classifier(x)
        return output


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import pandas as pd
import numpy
from keras.preprocessing import text, sequence
from sklearn import metrics
max_features = 5000
maxlen = 40
batch_size = 32
embedding_dims = 50


print('Loading data...')
df = pd.read_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/twitter16/twitter16.csv")
df = df.rename(columns = {'source_tweet':'message', 'label':'class'})
df = df.sample(frac=1)
train = df[:int(0.80*df.shape[0])]
test = df[int(0.80*df.shape[0]):]
print(train.columns)
print(test.columns)
x_train, y_train, x_test, y_test = train['message'], train['class'], test['message'], test['class']
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
# x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

embeddings_index = {}
for i, line in enumerate(open('/content/gdrive/MyDrive/Amazon/productner/data/glove.6B.100d.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

token = text.Tokenizer()
token.fit_on_texts(train['message'])
word_index = token.word_index

x_train = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=maxlen)
x_test = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=maxlen)

embedding_matrix = numpy.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
Index(['tweetID', 'message', 'class', 'tree'], dtype='object')
Index(['tweetID', 'message', 'class', 'tree'], dtype='object')
328 train sequences
83 test sequences
Pad sequences (samples x time)...
x_train shape: (328, 40)
x_test shape: (83, 40)


In [ ]:
epochs = 20
print('Build model...')
model = TextAttBiRNN(maxlen, max_features, embedding_dims)
model.compile('adam', loss = 'binary_crossentropy', metrics=['accuracy'])

Build model...


In [ ]:
print('Train...')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)

In [ ]:
result = (result.reshape(1,-1)[0]>0.5).astype(int)

In [ ]:
result

array([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1])

In [ ]:
print("ACC", metrics.accuracy_score(result, y_test))
print("Prec", metrics.precision_score(result, y_test))
print("REC", metrics.recall_score(result, y_test))
print("F1", metrics.f1_score(result, y_test))

ACC 0.9036144578313253
Prec 0.9230769230769231
REC 0.8780487804878049
F1 0.9


**Saving Trained Model**

In [ ]:
model.save_weights("_hcan_")

**Load Saved Model weights**

In [ ]:
model2 = HCAN(maxlen_sentence, maxlen_word, max_features, embedding_dims)
model2.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model2.load_weights("_hcan_")

In [ ]:
result = model2.predict(x_test)

In [ ]:
result = (result.reshape(1,-1)[0]>0.5).astype(int)
print("ACC", metrics.accuracy_score(result, y_test))
print("Prec", metrics.precision_score(result, y_test))
print("REC", metrics.recall_score(result, y_test))
print("F1", metrics.f1_score(result, y_test))

ACC 0.9824519230769231
Prec 0.9863272041489863
REC 0.9794007490636704
F1 0.982851773549448
